## OS Lab1

Multi-thread and multi-process matrix multiplication in python

## 基本矩陣乘法運算

$\begin{bmatrix}
a00 & a01
\\a10 & a11
\\a20 & a21
\end{bmatrix}
\times
\begin{bmatrix}
b00 & b01 & b02
\\b10 & b11 & b12
\end{bmatrix}
=
\begin{bmatrix}
ans00 & ans01 & ans02
\\ans10 & ans11 & ans12
\\ans20 & ans21 & ans22
\end{bmatrix}
\\
ans00=a00\times b00+a01\times b10
\\
ans01=a00\times b01+a01\times b11
\\
\vdots
$

## 切割運算過程
$\begin{bmatrix}
a00 & a01
\end{bmatrix}
\times
\begin{bmatrix}
b00 & b01 & b02
\\b10 & b11 & b12
\end{bmatrix}
=
\begin{bmatrix}
ans00 & ans01 & ans02
\end{bmatrix}
\\
\begin{bmatrix}
a10 & a11
\end{bmatrix}
\times
\begin{bmatrix}
b00 & b01 & b02
\\b10 & b11 & b12
\end{bmatrix}
=
\begin{bmatrix}
ans10 & ans11 & ans12
\end{bmatrix}
\\
\vdots
$

## 使用numpy作為容器

In [91]:
import numpy as np

def main():
    # Generate random matrix and result matrix
    matA = np.random.randint(10, size = (10, 10))
    matB = np.random.randint(10, size = (10, 10))
    result = np.zeros((matA.shape[0], matB.shape[1]))
    
    for row in range(0, matA.shape[0]):
        result[row] = np.matmul(matA[row], matB)

    # Compare with numpy's multiplication result
    print('Answer is correct:', np.all(np.matmul(matA, matB) == result))
    
if __name__ == "__main__":
    main()


Answer is correct: True


## Multi-thread in python
可用以下兩種方法：
1. 繼承threading.Thread並override run()
2. 生成Thread物件時將worker function及參數傳入

In [107]:
import threading

def thread_func(num1, num2):
    print(num1,'x',num2,'=',num1 * num2)

def main():
    # How many thread you want to use
    thread_num = 4
    threads = []

    # Assign job to threads
    for i in range(thread_num):
        # Pass argument to function with tuple
        thread = threading.Thread(target = thread_func, args = (i, 3))
        threads.append(thread)

    # run all threads
    for thread in threads:
        thread.start()

    # Wait for threads finish
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()


0 x 3 = 0
12 x 3 = 6
 x 3 = 3
3 x 3 = 9


## Global Interpreter Lock
每個Python Interpreter process僅佔用一個系統thread，因此不同python thread無法同時佔用CPU
因此只有程式不是CPU-bound時(例如寫入檔案/網路連線等IO-bound)
使用multi thread才能起到加速作用

## Multi-process in python
實做方法類似multi-thread，但由於不同process間彼此使用獨立的記憶體空間
因此需要使用`multiprocessing`提供的`Queue`來傳遞訊息

In [123]:
import multiprocessing
import random

def thread_func(process_no, result_queue):
    result_queue.put('Process ' + str(process_no) + ': ' + str(random.randint(0, 10)))

def main():
    # Generate queue for communication
    result_queue = multiprocessing.Manager().Queue()

    processes = 10
    jobs = []

    for i in range(processes):
        process = multiprocessing.Process(target = thread_func, args = (i, result_queue))
        jobs.append(process)

    for process in jobs:
        process.start()

    for process in jobs:
        process.join()

    while not result_queue.empty():
        result = result_queue.get()
        print(result)

if __name__ == "__main__":
    main()


Process 0: 7
Process 1: 1
Process 3: 1
Process 2: 0
Process 4: 3
Process 5: 4
Process 6: 7
Process 8: 3
Process 7: 8
Process 9: 8


## Timing in python
透過計算程式執行時間，可以估計使用multi-thread和multi-process造成的效能影響

In [140]:
import time

def main():
    start_time = time.time()
    
    a = 0
    for i in range(1000000):
        a = a + 3

    end_time = time.time()
    print('Time elapsed:\t', end_time - start_time)

if __name__ == "__main__":
    main()


Time elapsed:	 0.04612612724304199


## Lab1-矩陣平行運算
- 分別利用multi-thread和multi-process進行矩陣平行運算
- 需要和`numpy`計算的結果做比對，確保答案正確
- 需要計時，並和原本未使用thread或process加速的狀況做效能比較
- 隨機產生 10x10 / 100x100 / 1000x1000 的矩陣進行測試，比對不同計算量下的效能提昇